In [1]:
import sys 
sys.path.append(r"D:\Code\webui_demo")
from pinchoff_measurements import MDAC 

from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.instrument_drivers.QuTech.IVVI import IVVI

import numpy as np
import time

from qcodes import new_experiment, Station, Instrument
from qcodes.dataset.measurements import Measurement
from qcodes import ParamSpec

from pytopo.qctools import instruments as instools



C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from QT3_PCB.pad_conversion import Gen5_to_BoB

In [294]:
Gen5_to_BoB(58)

pad 58 is matrix element(1, 22)


[(1, 22)]

In [70]:
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
ami_x = instools.create_inst(AMI430, "AMI430_x", "192.168.0.157", port = 7180, has_current_rating=True)
ami_y = instools.create_inst(AMI430, "AMI430_y", "192.168.0.158", port = 7180, has_current_rating=True)
ami_z = instools.create_inst(AMI430, "AMI430_z", "192.168.0.159", port = 7180, has_current_rating=True)

ramp_limit_multiplier = 0.6
ami_x.current_ramp_limit(0.1802*ramp_limit_multiplier)
ami_x.coil_constant(0.01466)
ami_x.current_rating(68.21)

ami_y.current_ramp_limit(0.1784*ramp_limit_multiplier)
ami_y.coil_constant(0.01464)
ami_y.current_rating(68.31)

ami_z.current_ramp_limit(0.0775*ramp_limit_multiplier)
ami_z.coil_constant(0.0818)
ami_z.current_rating(73.35)

#ami = instools.create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)


Connected to: AMERICAN MAGNETICS INC. 430 (serial:170074, firmware:2.55) in 1.08s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170075, firmware:2.55) in 1.32s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170076, firmware:2.55) in 1.20s


D:\Code\Qcodes\qcodes\instrument_drivers\american_magnetics\AMI430.py:405: AMI430Warning: Increasing maximum ramp rate: we have a default current ramp rate limit of 0.06 A/s. We do not want to ramp faster than a set maximum so as to avoid quenching the magnet. A value of 0.06 A/s seems like a safe, conservative value for any magnet. Change this value at your own responsibility after consulting the specs of your particular magnet
  warn(warning_message, category=AMI430Warning)


In [72]:
ami_x.ramp_rate()

0.001

In [3]:
key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
sr = SR860("sr", "GPIB0::4::INSTR")
sr.amplitude.vals._min_value = 0
mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug=False)
ivvi = IVVI("ivvi", 'ASRL4::INSTR')

station = Station(sr, key, mdac, ivvi)

Connected to: Keysight Technologies 34465A (serial:MY57505052, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.86s
Connected to: Stanford_Research_Systems SR860 (serial:003223, firmware:V1.47) in 0.16s


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\ctwrapper\functions.py:1649: VisaIOWarning: VI_WARN_NSUP_ATTR_STATE (1073676420): Although the specified state of the attribute is valid, it is not supported by this implementation.
  return library.viSetAttribute(session, attribute, attribute_state)


Connected to: QNL Sydney MDAC (serial:0x32353532353551040028003E, firmware:1.4) in 5.08s
Initialized IVVI-rack in 0.05s


In [21]:
def set_mdac_in_safemode(nchan=64):
        sr.amplitude(0)
        sr.sine_outdc(0)
        ivvi.dac1(0)
        print("setting MDAC in save mode")
        
        for ch in mdac.channels[:nchan]:
            ch.voltage(0)
            #ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in mdac.channels[:nchan]:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')

In [22]:
set_mdac_in_safemode()

setting MDAC in save mode


In [7]:
ivvi.set_dacs_zero()

In [ ]:
def ramp(param, target, step_size, delay): 
    current_value = param()
    step_size *= np.sign(target - current_value)
    
    if abs(step_size) < 1e-10: 
        return 
    
    steps = np.arange(current_value, target, step_size)
    for step in steps: 
        param(step)
        time.sleep(delay)
    
    param(target)

gate_ramp_stepsize = 1e-3  #[V]
gate_ramp_delay = 0.01  #[s]

In [66]:
sr.amplitude(0.2)

In [8]:
ivvi.dac1()

0.03051804379333589

In [15]:
ivvi.dac1.post_delay

0

In [45]:
key.volt()

-0.15696137

### bias sweep

In [35]:
lockin_dc_ramp_stepsize=0.01  #[V]
lockin_dc_ramp_delay=1e-2  #[s]

def main():
    
    #g = np.array([0, -0.01])
    set_points =  np.linspace(0, -1, 71) #np.append(
    #    np.linspace(0, -4, 101),
    #    np.linspace(-4, 0, 101)
    #)

    config = {
        "rate": 2,  # V/s
        "rate_limit": 2,  # V/s
        "AC_exc_sample": 2e-5,  # TODO: Units?
        "gate_sweep_set_points": set_points, # [V]
        "gate_multiplier": 1,
        "AC_bias_multiplier": 1e-4,  # ISO-in attenuation + voltage source scale
        "DC_bias_multiplier": 1e-2,
        "awg_frequency": 21.359,  # Hz
        "awg_offset": 1e-3,  # V
        "AC_exc_zero": 0,  # V,
        "DC_exc_sample": np.linspace(0, 1e-3, 2),
        "current_gain": 1e6
    }

    config["AC_exc"] = config["AC_exc_sample"] / config["AC_bias_multiplier"]
    config["DC_exc"] = config["DC_exc_sample"][0] / config["DC_bias_multiplier"]
    
    # key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
    # sr = SR860("sr", "GPIB0::4::INSTR")
    # mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug = True)

    #print("making station")
    #station = Station(sr, key, mdac, ivvi)  # We do not add MDAC to prevent it being 
    # snapshot (takes a long time)
    #print("done")
    
    measurement_controller = IVVIAssistedMDACController(
        station,
        sources=[1],# and 6
        drain=57,
        gate=58,
        measurement_config=config
    )

    date = time.strftime("%Y-%m-%d")
    run_id = measurement_controller.run(f"{date}")
    # The experiments will be called Pinchoff_IVVIAssisted_<date> in the
    # data base and each experiment will have a data sets with names
    # "wire_1", "wire_2", etc.

class BaseMDACController:

    measurement_basename = ""

    def __init__(self, station, sources, drain, gate, measurement_config):
        """
        Args:
            station (qcodes.Station)
            sources (list): List of integers specifying source channels
            drain (int): Drain channel
            gate (int): Gate channel
            measurement_config (dict):
                Dictionary with measurement settings:
                * including rate limit
        """
        self._station = station
        self._mdac = station["MDAC1"]
        self._lockin = station['sr']
        self._keithley = station['key']
        self._ivvi = station['ivvi']

        self._sources = [self._mdac.channels[source - 1] for source in sources]
        self._drain = self._mdac.channels[drain - 1]
        self._gate = self._mdac.channels[gate - 1]
        self._measurement_config = measurement_config
        self._gate_set_points = None  # This is set by subclasses

        # The sweep delay is the integration time at each set point.
        # Note that this is different then the rate limit in the settings
        # dictionary as that determines how fast we go from one set point to
        # the next in V/s
        self._sweep_delay = self._lockin.time_constant() * 2

        self._set_mdac_in_safemode()

    def _set_mdac_in_safemode(self):
        
        print("setting MDAC in save mode")
        
        all_channels = self._sources + [self._drain, self._gate]
        self._lockin.amplitude(0)
        self._lockin.sine_outdc(0)
        self._ivvi.dac1(0)
        
        for ch in all_channels:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in all_channels:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')
        
        print("done")

    # This need to be in seperate subclasses
    def _setup(self):

        for chn in [self._drain, self._gate] + self._sources:
            chn.voltage(0)
            chn.gnd('open')
            chn.smc('close')
            chn.microd('open')
            chn.limit_rate(self._measurement_config["rate_limit"])

        self._drain.bus('open')
        self._drain.dac_output('open')
        self._drain.gnd('close')
        self._gate.smc('close')
        self._gate.dac_output('close')

        #for s in self._sources:
            #s.dac_output('open')

        #self._mdac.bus('close')

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass ")

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after finishing each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass")

    def run(self, measurement_name):
        self._setup()

        datasaver = None

        for source_num, source in enumerate(self._sources):

            exp = new_experiment(
                f"{self.measurement_basename}_{measurement_name}",
                sample_name=f"wire_{source_num}"
            )

            meas = Measurement(exp=exp, station=self._station)
            meas.register_custom_parameter(
                name="gate_voltage", unit="V"
            )
            
            meas.register_custom_parameter(
                name="bias_voltage", unit="V"
            )

            meas.register_custom_parameter(
                name="dI", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            
            meas.register_custom_parameter(
                name="I", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            

            meas.add_before_run(self._gate_sweep_setup, (source,))
            meas.add_after_run(self._gate_sweep_cleanup, (source,))

            biases = self._measurement_config["DC_exc_sample"] / self._measurement_config["DC_bias_multiplier"]
            
            # Run the sweep and save results in the data set
            with meas.run() as datasaver:
                for voltage in self._gate_set_points:
                    ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
                            gate_ramp_delay
                        )
                    
                    
                    for bias in  biases:

                        self._ivvi.dac1(bias*1e3)

                        time.sleep(self._sweep_delay)
                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
                        
                        datasaver.add_result(
                            ("gate_voltage", voltage),
                            ("bias_voltage", float(bias)),
                            ("dI", x_ac), 
                            ("I", x_dc)
                        )
            
#            with meas.run() as datasaver:
#                for bias in  biases:
#                    self._lockin.sine_outdc(bias)
#                    
#                    for voltage in self._gate_set_points:
#
#                        ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
#                            gate_ramp_delay
#                        )
#
#                        time.sleep(self._sweep_delay)
#                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
#                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
#                        
#                        datasaver.add_result(
#                            ("gate_voltage", voltage),
#                            ("bias_voltage", float(bias)),
#                            ("dI", x_ac), 
#                            ("I", x_dc)
#                        )
                        
        self._set_mdac_in_safemode()
        
        return datasaver.run_id


class IVVIAssistedMDACController(BaseMDACController):
    measurement_basename = "Pinchoff_IVVIAssisted"

    def __init__(self, station, sources, drain, gate, measurement_config):
        super().__init__(station, sources, drain, gate, measurement_config)

        set_points = self._measurement_config["gate_sweep_set_points"]
        multiplier = self._measurement_config["gate_multiplier"]

        self._gate_set_points = set_points / multiplier

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep.
        """
        #src.gnd('close')
        self._lockin.frequency(self._measurement_config["awg_frequency"])
        self._lockin.amplitude(self._measurement_config["AC_exc"])
        
        #self._ivvi.dac1()
        #ramp(
            #self._lockin.sine_outdc, 
            #self._measurement_config["DC_exc"], 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after running each gate sweep. We switch to the
        next source after this
        """
        ramp(self._gate.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        self._lockin.amplitude(self._measurement_config["AC_exc_zero"])
        
        self._ivvi.dac1(0)
        #ramp(
            #self._lockin.sine_outdc, 
            #0, 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)
        
        src.gnd('open')

In [36]:
main()

setting MDAC in save mode
done
Starting experimental run with id: 265
setting MDAC in save mode
done


In [16]:
lockin_dc_ramp_stepsize=0.01  #[V]
lockin_dc_ramp_delay=1e-2  #[s]

def main():
    
    set_points =  np.linspace(0, -1, 71) #np.append(
    #    np.linspace(0, -4, 101),
    #    np.linspace(-4, 0, 101)
    #)

    config = {
        "rate": 2,  # V/s
        "rate_limit": 2,  # V/s
        "AC_exc_sample": 2e-5,  # TODO: Units?
        "gate_sweep_set_points": set_points, # [V]
        "gate_multiplier": 1,
        "AC_bias_multiplier": 1e-4,  # ISO-in attenuation + voltage source scale
        "DC_bias_multiplier": 1e-2,
        "awg_frequency": 21.359,  # Hz
        "awg_offset": 1e-3,  # V
        "AC_exc_zero": 0,  # V,
        "DC_exc_sample": np.linspace(0, 1e-3, 2),
        "current_gain": 1e6
    }

    config["AC_exc"] = config["AC_exc_sample"] / config["AC_bias_multiplier"]
    config["DC_exc"] = config["DC_exc_sample"][0] / config["DC_bias_multiplier"]
    
    # key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
    # sr = SR860("sr", "GPIB0::4::INSTR")
    # mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug = True)

    #print("making station")
    #station = Station(sr, key, mdac, ivvi)  # We do not add MDAC to prevent it being 
    # snapshot (takes a long time)
    #print("done")
    
    measurement_controller = IVVIAssistedMDACController(
        station,
        sources=[1],# and 6
        drain=57,
        gate=58,
        measurement_config=config
    )

    date = time.strftime("%Y-%m-%d")
    run_id = measurement_controller.run(f"{date}")
    # The experiments will be called Pinchoff_IVVIAssisted_<date> in the
    # data base and each experiment will have a data sets with names
    # "wire_1", "wire_2", etc.

class BaseMDACController:

    measurement_basename = ""

    def __init__(self, station, sources, drain, gate, measurement_config):
        """
        Args:
            station (qcodes.Station)
            sources (list): List of integers specifying source channels
            drain (int): Drain channel
            gate (int): Gate channel
            measurement_config (dict):
                Dictionary with measurement settings:
                * including rate limit
        """
        self._station = station
        self._mdac = station["MDAC1"]
        self._lockin = station['sr']
        self._keithley = station['key']
        self._ivvi = station['ivvi']

        self._sources = [self._mdac.channels[source - 1] for source in sources]
        self._drain = self._mdac.channels[drain - 1]
        self._gate = self._mdac.channels[gate - 1]
        self._measurement_config = measurement_config
        self._gate_set_points = None  # This is set by subclasses

        # The sweep delay is the integration time at each set point.
        # Note that this is different then the rate limit in the settings
        # dictionary as that determines how fast we go from one set point to
        # the next in V/s
        self._sweep_delay = self._lockin.time_constant() * 2

        self._set_mdac_in_safemode()

    def _set_mdac_in_safemode(self):
        
        print("setting MDAC in save mode")
        
        all_channels = self._sources + [self._drain, self._gate]
        self._lockin.amplitude(0)
        self._lockin.sine_outdc(0)
        self._ivvi.dac1(0)
        
        for ch in all_channels:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in all_channels:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')
        
        print("done")

    # This need to be in seperate subclasses
    def _setup(self):

        for chn in [self._drain, self._gate] + self._sources:
            chn.voltage(0)
            chn.gnd('open')
            chn.smc('close')
            chn.microd('open')
            chn.limit_rate(self._measurement_config["rate_limit"])

        self._drain.bus('open')
        self._drain.dac_output('open')
        self._drain.gnd('close')
        self._gate.smc('close')
        self._gate.dac_output('close')

        #for s in self._sources:
            #s.dac_output('open')

        #self._mdac.bus('close')

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass ")

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after finishing each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass")

    def run(self, measurement_name):
        self._setup()

        datasaver = None

        for source_num, source in enumerate(self._sources):

            exp = new_experiment(
                f"{self.measurement_basename}_{measurement_name}",
                sample_name=f"wire_{source_num}"
            )

            meas = Measurement(exp=exp, station=self._station)
            meas.register_custom_parameter(
                name="gate_voltage", unit="V"
            )
            
            meas.register_custom_parameter(
                name="bias_voltage", unit="V"
            )

            meas.register_custom_parameter(
                name="dI", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            
            meas.register_custom_parameter(
                name="I", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            

            meas.add_before_run(self._gate_sweep_setup, (source,))
            meas.add_after_run(self._gate_sweep_cleanup, (source,))

            biases = self._measurement_config["DC_exc_sample"] / self._measurement_config["DC_bias_multiplier"]
            
            # Run the sweep and save results in the data set
            with meas.run() as datasaver:
                for voltage in self._gate_set_points:
                    ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
                            gate_ramp_delay
                        )
                    
                    
                    for bias in  biases:

                        self._ivvi.dac1(bias*1e3)

                        time.sleep(self._sweep_delay)
                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
                        
                        datasaver.add_result(
                            ("gate_voltage", voltage),
                            ("bias_voltage", float(bias)),
                            ("dI", x_ac), 
                            ("I", x_dc)
                        )
            
#            with meas.run() as datasaver:
#                for bias in  biases:
#                    self._lockin.sine_outdc(bias)
#                    
#                    for voltage in self._gate_set_points:
#
#                        ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
#                            gate_ramp_delay
#                        )
#
#                        time.sleep(self._sweep_delay)
#                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
#                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
#                        
#                        datasaver.add_result(
#                            ("gate_voltage", voltage),
#                            ("bias_voltage", float(bias)),
#                            ("dI", x_ac), 
#                            ("I", x_dc)
#                        )
                        
        self._set_mdac_in_safemode()
        
        return datasaver.run_id


class IVVIAssistedMDACController(BaseMDACController):
    measurement_basename = "Pinchoff_IVVIAssisted"

    def __init__(self, station, sources, drain, gate, measurement_config):
        super().__init__(station, sources, drain, gate, measurement_config)

        set_points = self._measurement_config["gate_sweep_set_points"]
        multiplier = self._measurement_config["gate_multiplier"]

        self._gate_set_points = set_points / multiplier

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep.
        """
        #src.gnd('close')
        self._lockin.frequency(self._measurement_config["awg_frequency"])
        self._lockin.amplitude(self._measurement_config["AC_exc"])
        
        #self._ivvi.dac1()
        #ramp(
            #self._lockin.sine_outdc, 
            #self._measurement_config["DC_exc"], 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after running each gate sweep. We switch to the
        next source after this
        """
        ramp(self._gate.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        self._lockin.amplitude(self._measurement_config["AC_exc_zero"])
        
        self._ivvi.dac1(0)
        #ramp(
            #self._lockin.sine_outdc, 
            #0, 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)
        
        src.gnd('open')

In [18]:
main()

setting MDAC in save mode
done
Starting experimental run with id: 264


KeyboardInterrupt: 

In [116]:
def ramp(param, target, step_size, delay): 
    current_value = param()
    step_size *= np.sign(target - current_value)
    
    if abs(step_size) < 1e-10: 
        return 
    
    steps = np.arange(current_value, target, step_size)
    for step in steps: 
        param(step)
        time.sleep(delay)
    
    param(target)

gate_ramp_stepsize = 10e-3  #[V]
gate_ramp_delay = 2/(5/gate_ramp_stepsize)  #[s]  
# If we are at 5V, we want to ramp to zero in 2 seconds. 

lockin_dc_ramp_stepsize=0.01  #[V]
lockin_dc_ramp_delay=1e-2  #[s]

def main():
    
    set_points =  np.linspace(0, -3.5, 71) #np.append(
    #    np.linspace(0, -4, 101),
    #    np.linspace(-4, 0, 101)
    #)

    config = {
        "rate": 2,  # V/s
        "rate_limit": 2,  # V/s
        "AC_exc_sample": 2e-5,  # TODO: Units?
        "gate_sweep_set_points": set_points, # [V]
        "gate_multiplier": 1,
        "AC_bias_multiplier": 1e-4,  # ISO-in attenuation + voltage source scale
        "DC_bias_multiplier": 1e-2,
        "awg_frequency": 21.359,  # Hz
        "awg_offset": 1e-3,  # V
        "AC_exc_zero": 0,  # V,
        "DC_exc_sample": np.linspace(-15e-3, 15e-3, 61),
        "current_gain": 1e6
    }

    config["AC_exc"] = config["AC_exc_sample"] / config["AC_bias_multiplier"]
    config["DC_exc"] = config["DC_exc_sample"][0] / config["DC_bias_multiplier"]
    
    # key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
    # sr = SR860("sr", "GPIB0::4::INSTR")
    # mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug = True)

    #print("making station")
    #station = Station(sr, key, mdac, ivvi)  # We do not add MDAC to prevent it being 
    # snapshot (takes a long time)
    #print("done")
    
    measurement_controller = IVVIAssistedMDACController(
        station,
        sources=[23],# and 6
        drain=24,
        gate=64,
        measurement_config=config
    )

    date = time.strftime("%Y-%m-%d")
    run_id = measurement_controller.run(f"{date}")
    # The experiments will be called Pinchoff_IVVIAssisted_<date> in the
    # data base and each experiment will have a data sets with names
    # "wire_1", "wire_2", etc.

class BaseMDACController:

    measurement_basename = ""

    def __init__(self, station, sources, drain, gate, measurement_config):
        """
        Args:
            station (qcodes.Station)
            sources (list): List of integers specifying source channels
            drain (int): Drain channel
            gate (int): Gate channel
            measurement_config (dict):
                Dictionary with measurement settings:
                * including rate limit
        """
        self._station = station
        self._mdac = station["MDAC1"]
        self._lockin = station['sr']
        self._keithley = station['key']
        self._ivvi = station['ivvi']

        self._sources = [self._mdac.channels[source - 1] for source in sources]
        self._drain = self._mdac.channels[drain - 1]
        self._gate = self._mdac.channels[gate - 1]
        self._measurement_config = measurement_config
        self._gate_set_points = None  # This is set by subclasses

        # The sweep delay is the integration time at each set point.
        # Note that this is different then the rate limit in the settings
        # dictionary as that determines how fast we go from one set point to
        # the next in V/s
        self._sweep_delay = self._lockin.time_constant() * 4

        self._set_mdac_in_safemode()

    def _set_mdac_in_safemode(self):
        
        print("setting MDAC in save mode")
        
        all_channels = self._sources + [self._drain, self._gate]
        self._lockin.amplitude(0)
        self._lockin.sine_outdc(0)
        self._ivvi.dac1(0)
        
        for ch in all_channels:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in all_channels:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')
        
        print("done")

    # This need to be in seperate subclasses
    def _setup(self):

        for chn in [self._drain, self._gate] + self._sources:
            chn.voltage(0)
            chn.gnd('open')
            chn.smc('open')
            chn.microd('close')
            chn.limit_rate(self._measurement_config["rate_limit"])

        self._drain.bus('close')
        self._drain.dac_output('open')
        self._gate.smc('close')
        self._gate.dac_output('close')

        for s in self._sources:
            s.dac_output('open')

        self._mdac.bus('close')

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass ")

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after finishing each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass")

    def run(self, measurement_name):
        self._setup()

        datasaver = None

        for source_num, source in enumerate(self._sources):

            exp = new_experiment(
                f"{self.measurement_basename}_{measurement_name}",
                sample_name=f"wire_{source_num}"
            )

            meas = Measurement(exp=exp, station=self._station)
            meas.register_custom_parameter(
                name="gate_voltage", unit="V"
            )
            
            meas.register_custom_parameter(
                name="bias_voltage", unit="V"
            )

            meas.register_custom_parameter(
                name="dI", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            
            meas.register_custom_parameter(
                name="I", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            

            meas.add_before_run(self._gate_sweep_setup, (source,))
            meas.add_after_run(self._gate_sweep_cleanup, (source,))

            biases = self._measurement_config["DC_exc_sample"] / self._measurement_config["DC_bias_multiplier"]
            
            # Run the sweep and save results in the data set
            with meas.run() as datasaver:
                for voltage in self._gate_set_points:
                    ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
                            gate_ramp_delay
                        )
                    
                    
                    for bias in  biases:

                        self._ivvi.dac1(bias*1e3)

                        time.sleep(self._sweep_delay)
                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
                        
                        datasaver.add_result(
                            ("gate_voltage", voltage),
                            ("bias_voltage", float(bias)),
                            ("dI", x_ac), 
                            ("I", x_dc)
                        )
            
#            with meas.run() as datasaver:
#                for bias in  biases:
#                    self._lockin.sine_outdc(bias)
#                    
#                    for voltage in self._gate_set_points:
#
#                        ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
#                            gate_ramp_delay
#                        )
#
#                        time.sleep(self._sweep_delay)
#                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
#                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
#                        
#                        datasaver.add_result(
#                            ("gate_voltage", voltage),
#                            ("bias_voltage", float(bias)),
#                            ("dI", x_ac), 
#                            ("I", x_dc)
#                        )
                        
        self._set_mdac_in_safemode()
        
        return datasaver.run_id


class IVVIAssistedMDACController(BaseMDACController):
    measurement_basename = "Pinchoff_IVVIAssisted"

    def __init__(self, station, sources, drain, gate, measurement_config):
        super().__init__(station, sources, drain, gate, measurement_config)

        set_points = self._measurement_config["gate_sweep_set_points"]
        multiplier = self._measurement_config["gate_multiplier"]

        self._gate_set_points = set_points / multiplier

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep.
        """
        src.gnd('close')
        self._lockin.frequency(self._measurement_config["awg_frequency"])
        self._lockin.amplitude(self._measurement_config["AC_exc"])
        
        #self._ivvi.dac1()
        #ramp(
            #self._lockin.sine_outdc, 
            #self._measurement_config["DC_exc"], 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after running each gate sweep. We switch to the
        next source after this
        """
        ramp(self._gate.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        self._lockin.amplitude(self._measurement_config["AC_exc_zero"])
        
        self._ivvi.dac1(0)
        #ramp(
            #self._lockin.sine_outdc, 
            #0, 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)
        
        src.gnd('open')

In [117]:
main()

setting MDAC in save mode
done
Starting experimental run with id: 236
setting MDAC in save mode
done


In [215]:
sr.amplitude(0)
ivvi.dac1(0)
set_mdac_in_safemode()

setting MDAC in save mode


In [236]:
ivvi.dac1.step

10

In [239]:
ivvi.dac1.inter_delay

0.1

In [242]:
ivvi.dac1.post_delay

0.1

In [165]:
sr.amplitude(0)
#mdac.ch58.dac_output()

In [166]:
sr.X()

1.6411403294e-06

In [129]:
ramp(mdac.ch58.voltage, 0, gate_ramp_stepsize, 
                            gate_ramp_delay)

In [127]:
mdac.ch58.('close')

### QPC 0deg40nm
#### Gate 1-12 --> 58
#### Drain 1-11 --> 57
#### S2 2-25 --> 59

In [225]:
def ramp(param, target, step_size, delay): 
    current_value = param()
    step_size *= np.sign(target - current_value)
    
    if abs(step_size) < 1e-10: 
        return 
    
    steps = np.arange(current_value, target, step_size)
    for step in steps: 
        param(step)
        time.sleep(delay)
    
    param(target)

gate_ramp_stepsize = 5e-3  #[V]
gate_ramp_delay = 0.1 #2/(5/gate_ramp_stepsize)  #[s]  
# If we are at 5V, we want to ramp to zero in 2 seconds. 

lockin_dc_ramp_stepsize=0.01  #[V]
lockin_dc_ramp_delay=1e-2  #[s]

b = np.array([0.25e-3, 1e-3, 10e-3])

def main():
    
    set_points =  np.linspace(-1.25, -1.75, 101) #np.append(
    #    np.linspace(0, -4, 101),
    #    np.linspace(-4, 0, 101)
    #)

    config = {
        "rate": 2,  # V/s
        "rate_limit": 2,  # V/s
        "AC_exc_sample": 2e-5,  # TODO: Units?
        "gate_sweep_set_points": set_points, # [V]
        "gate_multiplier": 1,
        "AC_bias_multiplier": 1e-4,  # ISO-in attenuation + voltage source scale
        "DC_bias_multiplier": 1e-2,
        "awg_frequency": 13.2040,  # Hz
        "awg_offset": 1e-3,  # V
        "AC_exc_zero": 0,  # V,
        "DC_exc_sample": b, #np.linspace(-2e-3, 2e-3, 41),
        "current_gain": 1e6
    }

    config["AC_exc"] = config["AC_exc_sample"] / config["AC_bias_multiplier"]
    config["DC_exc"] = config["DC_exc_sample"][0] / config["DC_bias_multiplier"]
    
    # key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
    # sr = SR860("sr", "GPIB0::4::INSTR")
    # mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug = True)

    #print("making station")
    #station = Station(sr, key, mdac, ivvi)  # We do not add MDAC to prevent it being 
    # snapshot (takes a long time)
    #print("done")
    
    measurement_controller = IVVIAssistedMDACController(
        station,
        sources=[59],
        drain=57,
        gate=58,
        measurement_config=config
    )

    date = time.strftime("%Y-%m-%d")
    run_id = measurement_controller.run(f"{date}")
    # The experiments will be called Pinchoff_IVVIAssisted_<date> in the
    # data base and each experiment will have a data sets with names
    # "wire_1", "wire_2", etc.

class BaseMDACController:

    measurement_basename = ""

    def __init__(self, station, sources, drain, gate, measurement_config):
        """
        Args:
            station (qcodes.Station)
            sources (list): List of integers specifying source channels
            drain (int): Drain channel
            gate (int): Gate channel
            measurement_config (dict):
                Dictionary with measurement settings:
                * including rate limit
        """
        self._station = station
        self._mdac = station["MDAC1"]
        self._lockin = station['sr']
        self._keithley = station['key']
        self._ivvi = station['ivvi']

        self._sources = [self._mdac.channels[source - 1] for source in sources]
        self._drain = self._mdac.channels[drain - 1]
        self._gate = self._mdac.channels[gate - 1]
        self._measurement_config = measurement_config
        self._gate_set_points = None  # This is set by subclasses

        # The sweep delay is the integration time at each set point.
        # Note that this is different then the rate limit in the settings
        # dictionary as that determines how fast we go from one set point to
        # the next in V/s
        self._sweep_delay = self._lockin.time_constant() * 2

        self._set_mdac_in_safemode()

    def _set_mdac_in_safemode(self):
        
        print("setting MDAC in save mode")
        
        all_channels = self._sources + [self._drain, self._gate]
        self._lockin.amplitude(0)
        self._lockin.sine_outdc(0)
        self._ivvi.dac1(0)
        
        for ch in all_channels:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in all_channels:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')
        
        print("done")

    # This need to be in seperate subclasses
    def _setup(self):

        for chn in [self._drain, self._gate] + self._sources:
            chn.voltage(0)
            chn.gnd('open')
            chn.smc('open')
            chn.microd('close')
            chn.limit_rate(self._measurement_config["rate_limit"])

        self._drain.microd('open')
        self._drain.bus('close')
        self._drain.dac_output('open')
        self._drain.smc('close')
        
        self._gate.smc('close')
        self._gate.dac_output('close')

        for s in self._sources:
            s.microd('open')
            s.dac_output('open')
            s.smc('close')

        self._mdac.bus('close')

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass ")

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after finishing each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass")

    def run(self, measurement_name):
        self._setup()

        datasaver = None

        for source_num, source in enumerate(self._sources):

            exp = new_experiment(
                f"{self.measurement_basename}_{measurement_name}",
                sample_name=f"wire_{source_num}"
            )

            meas = Measurement(exp=exp, station=self._station)
            meas.register_custom_parameter(
                name="gate_voltage", unit="V"
            )
            
            meas.register_custom_parameter(
                name="bias_voltage", unit="V"
            )

            meas.register_custom_parameter(
                name="dI", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            
            meas.register_custom_parameter(
                name="I", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            

            meas.add_before_run(self._gate_sweep_setup, (source,))
            meas.add_after_run(self._gate_sweep_cleanup, (source,))

            biases = self._measurement_config["DC_exc_sample"] / self._measurement_config["DC_bias_multiplier"]
            
            # Run the sweep and save results in the data set
            with meas.run() as datasaver:
                for voltage in self._gate_set_points:
                    ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
                            gate_ramp_delay
                        )
                    
                    
                    for bias in  biases:

                        self._ivvi.dac1(bias*1e3)

                        time.sleep(self._sweep_delay)
                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
                        
                        datasaver.add_result(
                            ("gate_voltage", voltage),
                            ("bias_voltage", float(bias)),
                            ("dI", x_ac), 
                            ("I", x_dc)
                        )
            
#            with meas.run() as datasaver:
#                for bias in  biases:
#                    self._lockin.sine_outdc(bias)
#                    
#                    for voltage in self._gate_set_points:
#
#                        ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
#                            gate_ramp_delay
#                        )
#
#                        time.sleep(self._sweep_delay)
#                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
#                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
#                        
#                        datasaver.add_result(
#                            ("gate_voltage", voltage),
#                            ("bias_voltage", float(bias)),
#                            ("dI", x_ac), 
#                            ("I", x_dc)
#                        )
                        
        self._set_mdac_in_safemode()
        
        return datasaver.run_id


class IVVIAssistedMDACController(BaseMDACController):
    measurement_basename = "Pinchoff_IVVIAssisted"

    def __init__(self, station, sources, drain, gate, measurement_config):
        super().__init__(station, sources, drain, gate, measurement_config)

        set_points = self._measurement_config["gate_sweep_set_points"]
        multiplier = self._measurement_config["gate_multiplier"]

        self._gate_set_points = set_points / multiplier

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep.
        """
        src.gnd('close')
        self._lockin.frequency(self._measurement_config["awg_frequency"])
        self._lockin.amplitude(self._measurement_config["AC_exc"])
        
        #self._ivvi.dac1()
        #ramp(
            #self._lockin.sine_outdc, 
            #self._measurement_config["DC_exc"], 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after running each gate sweep. We switch to the
        next source after this
        """
        ramp(self._gate.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        self._lockin.amplitude(self._measurement_config["AC_exc_zero"])
        
        self._ivvi.dac1(0)
        #ramp(
            #self._lockin.sine_outdc, 
            #0, 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)
        
        src.gnd('open')

In [226]:
main()

setting MDAC in save mode
done
Starting experimental run with id: 253
setting MDAC in save mode
done


In [224]:
#ami_x.field(0)
ami_z.field()

-2.32e-05

In [252]:
ivvi.dac1.step

1

In [273]:
sr.amplitude(0)
ivvi.dac1(0)
set_mdac_in_safemode()

setting MDAC in save mode


### QPC 0deg40nm
#### Gate 1-12 --> 58
#### Drain 1-11 --> 57
#### S3 2-14 --> 60

In [271]:
def ramp(param, target, step_size, delay): 
    current_value = param()
    step_size *= np.sign(target - current_value)
    
    if abs(step_size) < 1e-10: 
        return 
    
    steps = np.arange(current_value, target, step_size)
    for step in steps: 
        param(step)
        time.sleep(delay)
    
    param(target)

gate_ramp_stepsize = 1e-3  #[V]
gate_ramp_delay = 0.15 #2/(5/gate_ramp_stepsize)  #[s]  
# If we are at 5V, we want to ramp to zero in 2 seconds. 

lockin_dc_ramp_stepsize=0.01  #[V]
lockin_dc_ramp_delay=1e-2  #[s]

b = np.array([0.025e-3, 0.1e-3])

def main():
    
    set_points =  np.linspace(-1.2, -2.2, 201) #np.append(
    #    np.linspace(0, -4, 101),
    #    np.linspace(-4, 0, 101)
    #)

    config = {
        "rate": 2,  # V/s
        "rate_limit": 2,  # V/s
        "AC_exc_sample": 2e-5,  # TODO: Units?
        "gate_sweep_set_points": set_points, # [V]
        "gate_multiplier": 1,
        "AC_bias_multiplier": 1e-4,  # ISO-in attenuation + voltage source scale
        "DC_bias_multiplier": 1e-2,
        "awg_frequency": 13.2040,  # Hz
        "awg_offset": 1e-3,  # V
        "AC_exc_zero": 0,  # V,
        "DC_exc_sample": 1e-3, #np.linspace(-5e-3, 5e-3, 41),
        "current_gain": 1e6
    }

    config["AC_exc"] = config["AC_exc_sample"] / config["AC_bias_multiplier"]
    config["DC_exc"] = config["DC_exc_sample"] / config["DC_bias_multiplier"]  #[0]
    
    # key = Keysight_34465A("key", "TCPIP0::192.168.0.150")
    # sr = SR860("sr", "GPIB0::4::INSTR")
    # mdac = MDAC('MDAC1', 'ASRL5::INSTR', debug = True)

    #print("making station")
    #station = Station(sr, key, mdac, ivvi)  # We do not add MDAC to prevent it being 
    # snapshot (takes a long time)
    #print("done")
    
    measurement_controller = IVVIAssistedMDACController(
        station,
        sources=[60],
        drain=57,
        gate=58,
        measurement_config=config
    )

    date = time.strftime("%Y-%m-%d")
    run_id = measurement_controller.run(f"{date}")
    # The experiments will be called Pinchoff_IVVIAssisted_<date> in the
    # data base and each experiment will have a data sets with names
    # "wire_1", "wire_2", etc.

class BaseMDACController:

    measurement_basename = ""

    def __init__(self, station, sources, drain, gate, measurement_config):
        """
        Args:
            station (qcodes.Station)
            sources (list): List of integers specifying source channels
            drain (int): Drain channel
            gate (int): Gate channel
            measurement_config (dict):
                Dictionary with measurement settings:
                * including rate limit
        """
        self._station = station
        self._mdac = station["MDAC1"]
        self._lockin = station['sr']
        self._keithley = station['key']
        self._ivvi = station['ivvi']

        self._sources = [self._mdac.channels[source - 1] for source in sources]
        self._drain = self._mdac.channels[drain - 1]
        self._gate = self._mdac.channels[gate - 1]
        self._measurement_config = measurement_config
        self._gate_set_points = None  # This is set by subclasses

        # The sweep delay is the integration time at each set point.
        # Note that this is different then the rate limit in the settings
        # dictionary as that determines how fast we go from one set point to
        # the next in V/s
        self._sweep_delay = self._lockin.time_constant() * 2

        self._set_mdac_in_safemode()

    def _set_mdac_in_safemode(self):
        
        print("setting MDAC in save mode")
        
        all_channels = self._sources + [self._drain, self._gate]
        self._lockin.amplitude(0)
        self._lockin.sine_outdc(0)
        self._ivvi.dac1(0)
        
        for ch in all_channels:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in all_channels:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')
        
        print("done")

    # This need to be in seperate subclasses
    def _setup(self):

        for chn in [self._drain, self._gate] + self._sources:
            chn.voltage(0)
            chn.gnd('open')
            chn.smc('open')
            chn.microd('close')
            chn.limit_rate(self._measurement_config["rate_limit"])

        self._drain.microd('open')
        self._drain.bus('close')
        self._drain.dac_output('open')
        self._drain.smc('close')
        
        self._gate.smc('close')
        self._gate.dac_output('close')

        for s in self._sources:
            s.microd('open')
            s.dac_output('open')
            s.smc('close')

        self._mdac.bus('close')

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass ")

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after finishing each gate sweep. This is different
        depending on if we use the IVVI or not, hence not implemented in base
        """
        raise NotImplementedError("Please subclass")

    def run(self, measurement_name):
        self._setup()

        datasaver = None

        for source_num, source in enumerate(self._sources):

            exp = new_experiment(
                f"{self.measurement_basename}_{measurement_name}",
                sample_name=f"wire_{source_num}"
            )

            meas = Measurement(exp=exp, station=self._station)
            meas.register_custom_parameter(
                name="gate_voltage", unit="V"
            )
            
            meas.register_custom_parameter(
                name="bias_voltage", unit="V"
            )

            meas.register_custom_parameter(
                name="dI", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            
            meas.register_custom_parameter(
                name="I", unit="A", setpoints=("gate_voltage", "bias_voltage")
            )
            

            meas.add_before_run(self._gate_sweep_setup, (source,))
            meas.add_after_run(self._gate_sweep_cleanup, (source,))

            biases = self._measurement_config["DC_exc_sample"] / self._measurement_config["DC_bias_multiplier"]
            
            # Run the sweep and save results in the data set
            with meas.run() as datasaver:
                self._ivvi.dac1(biases*1e3)
                for voltage in self._gate_set_points:
                    ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
                            gate_ramp_delay
                        )
                    
                    
                    #for bias in  biases:

                        #self._ivvi.dac1(bias*1e3)

                    time.sleep(self._sweep_delay)
                    x_ac = self._lockin.X() / self._measurement_config["current_gain"]
                    x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]

                    datasaver.add_result(
                        ("gate_voltage", voltage),
                        ("bias_voltage", float(biases)),
                        ("dI", x_ac), 
                        ("I", x_dc)
                    )
            
#            with meas.run() as datasaver:
#                for bias in  biases:
#                    self._lockin.sine_outdc(bias)
#                    
#                    for voltage in self._gate_set_points:
#
#                        ramp(self._gate.voltage, voltage, gate_ramp_stepsize, 
#                            gate_ramp_delay
#                        )
#
#                        time.sleep(self._sweep_delay)
#                        x_ac = self._lockin.X() / self._measurement_config["current_gain"]
#                        x_dc = - self._keithley.volt() / self._measurement_config["current_gain"]
#                        
#                        datasaver.add_result(
#                            ("gate_voltage", voltage),
#                            ("bias_voltage", float(bias)),
#                            ("dI", x_ac), 
#                            ("I", x_dc)
#                        )
                        
        self._set_mdac_in_safemode()
        
        return datasaver.run_id


class IVVIAssistedMDACController(BaseMDACController):
    measurement_basename = "Pinchoff_IVVIAssisted"

    def __init__(self, station, sources, drain, gate, measurement_config):
        super().__init__(station, sources, drain, gate, measurement_config)

        set_points = self._measurement_config["gate_sweep_set_points"]
        multiplier = self._measurement_config["gate_multiplier"]

        self._gate_set_points = set_points / multiplier

    def _gate_sweep_setup(self, src):
        """
        This method is run before starting each gate sweep.
        """
        src.gnd('close')
        self._lockin.frequency(self._measurement_config["awg_frequency"])
        self._lockin.amplitude(self._measurement_config["AC_exc"])
        
        #self._ivvi.dac1()
        #ramp(
            #self._lockin.sine_outdc, 
            #self._measurement_config["DC_exc"], 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)

    def _gate_sweep_cleanup(self, src):
        """
        This method is run after running each gate sweep. We switch to the
        next source after this
        """
        ramp(self._gate.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        self._lockin.amplitude(self._measurement_config["AC_exc_zero"])
        
        self._ivvi.dac1(0)
        #ramp(
            #self._lockin.sine_outdc, 
            #0, 
            #lockin_dc_ramp_stepsize, 
            #lockin_dc_ramp_delay
        #)
        
        src.gnd('open')

In [272]:
main()

setting MDAC in save mode
done
Starting experimental run with id: 263
setting MDAC in save mode
done


In [113]:
ivvi.dac1()

0.03051804379333589

In [114]:
sr.sine_outdc(0)
sr.amplitude(0)

In [6]:
sr.time_constant()

0.1

In [76]:
def set_mdac_in_safemode(nchan=64):
        
        print("setting MDAC in save mode")
        
        for ch in mdac.channels[:nchan]:
            ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

        # Note: we do this in separate loops because we want to be sure that
        # no currents are running anywhere before switching the relays
        for ch in mdac.channels[:nchan]:
            ch.dac_output('open')
            ch.bus('open')
            ch.gnd('open')
            ch.smc('open')
            ch.microd('open')
            ch.divider('off')

In [115]:
set_mdac_in_safemode()

setting MDAC in save mode


In [16]:
mdac.ch64.voltage()

0.0

In [7]:
sr.amplitude()

0.0099999997765

In [5]:

def relay_states(ch):
    print('ch: ',ch.channel_number(), 
          '\nsmc ',ch.smc(), 
          '\ngnd ', ch.gnd(), 
          '\nud ', ch.microd(),
          '\ndac ', ch.dac_output(), 
          '\nbus ',ch.bus())

In [13]:
ch23 = mdac.ch23
ch23.microd('close')
ch23.smc('open')
ch23.gnd('close')
relay_states(ch23)

ch24 = mdac.ch24
ch24.microd('close')
ch24.bus('close')
ch24.smc('open')
relay_states(ch24)
ch64 = mdac.ch64
relay_states(ch64)
mdac.bus('close')

ch:  23 
smc  open 
gnd  close 
ud  close 
dac  open 
bus  open
ch:  24 
smc  open 
gnd  open 
ud  close 
dac  open 
bus  close
ch:  64 
smc  close 
gnd  open 
ud  close 
dac  close 
bus  open


In [12]:
sr.amplitude(0)
ramp(ch64.voltage, 0, 0.01, 0.1)

In [42]:
sr.amplitude.vals._min_value = 0

In [43]:
sr.amplitude(0)

In [44]:
sr.amplitude()

0.0

In [47]:
sr.sine_outdc()

0.0